## RAG example with Langchain, Milvus, and vLLM

Requirements:
- A Milvus instance, either standalone or cluster.
- Connection credentials to Milvus must be available as environment variables: MILVUS_USERNAME and MILVUS_PASSWORD.
- A vLLM inference endpoint. In this example we use the OpenAI Compatible API.

### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0 openai==1.13.3


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import VLLMOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Milvus

#### Bases parameters, Inference server and Milvus info

In [3]:
# Replace values according to your Milvus deployment (including the proper Milvus Collection name)
INFERENCE_SERVER_URL = "http://vllm.vllm.svc.cluster.local:8000/v1"
MODEL_NAME = "Qwen/Qwen2-0.5B-Instruct"
MAX_TOKENS=1024
TOP_P=0.95
TEMPERATURE=0.01
PRESENCE_PENALTY=1.03
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "catalogo_ba_gov"

#### Initialize the connection

In [4]:
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/opt/app-root/src/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/40b98394640e630d5276807046089b233113aa87/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serial

#### Initialize query chain

In [5]:
template="""<s>[INST] <<SYS>>
Você é um assistente prestativo que deve apenas responder as perguntas fornecidas.

Você receberá uma pergunta que precisa responder, juntamente com um contexto que fornecerá informações. Você deve responder à pergunta com base nesse contexto.

Se uma pergunta não fizer sentido ou não for factualmente coerente, faça perguntas para entender o contexto, mas nao responda de forma alguma algo incorreto. Se você não souber a resposta para uma pergunta, não compartilhe informações falsas.
<</SYS>>

Context: 
{context}

Question: {question} [/INST]
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm =  VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=INFERENCE_SERVER_URL,
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    temperature=TEMPERATURE,
    presence_penalty=PRESENCE_PENALTY,
    streaming=True,
    verbose=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
            ),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
        )

os.environ["TOKENIZERS_PARALLELISM"] = "false"

#### Query example

In [8]:
question = "Qual o passo a passo para um bolo de cenoura"
result = qa_chain.invoke({"query": question})

<</SYS>>

Context: 
# SOLICITAR EMISSÃO DE 2ª VIA DE IPTU - LAURO DE FREITAS

#### Sobre
SOLICITAR EMISSÃO DE 2ª VIA DE IPTU - LAURO DE FREITAS

## Documentos Necessários

### DESCRIÇÃO DO SERVIÇO  
Serviço destinado aos cidadãos que desejam obter a Emissão de 2º Via de IPTU do município de Lauro de Freitas.  
  
O IPTU é um imposto que incide sobre a propriedade, domínio útil ou posse imobiliária urbana, tanto para pessoas físicas, como para jurídicas. Com alíquota definida em lei municipal e benefício de desconto, definido no calendário fiscal.  
  
### PRÉ-REQUISITO DO SERVIÇO

01. O Requerente deve possuir senha na PLATAFORMA BA.GOV.BR ou no PORTAL GOV.BR e efetuar o login.  
  
### QUEM PODE SOLICITAR  
01. Negócios/Empresas;  
02. Cidadão.  
  
### OUTROS NOMES PELOS QUAIS O SERVIÇO É CONHECIDO  
EMISSÃO DE SEGUNDA VIA DE IPTU 2024  
  
### AGENDAMENTO  
Não se aplica.

## Informações Adicionais

#### INFORMAÇÕES ADICIONAIS  
Não se aplica.  
  
#### PRAZO DE ENTREGA/CONCLUSÃO DO

#### Retrieve source

In [7]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)

markdown/IPTU.md
